In [180]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

### Import the dataset

In [181]:
df = pd.read_csv('flights_test_last.csv')

In [182]:
df

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01,WN,WN,WN,5774,WN,N8542Z,5774,15016,STL,"St. Louis, MO",13204,MCO,"Orlando, FL",1320,1635,N,135,1,882
1,2020-01-01,WN,WN,WN,4650,WN,N457WN,4650,15016,STL,"St. Louis, MO",13232,MDW,"Chicago, IL",1150,1300,N,70,1,251
2,2020-01-01,WN,WN,WN,4681,WN,N7727A,4681,15016,STL,"St. Louis, MO",13232,MDW,"Chicago, IL",1825,1930,N,65,1,251
3,2020-01-01,WN,WN,WN,5330,WN,N457WN,5330,15016,STL,"St. Louis, MO",13232,MDW,"Chicago, IL",800,905,N,65,1,251
4,2020-01-01,WN,WN,WN,5963,WN,N216WR,5963,15016,STL,"St. Louis, MO",13232,MDW,"Chicago, IL",2020,2130,N,70,1,251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31471,2020-01-31,AA,AA_CODESHARE,AA,4398,YX,N112HQ,4398,14100,PHL,"Philadelphia, PA",14576,ROC,"Rochester, NY",2120,2243,N,83,1,257
31472,2020-01-31,AA,AA_CODESHARE,AA,4399,YX,N128HQ,4399,11057,CLT,"Charlotte, NC",12339,IND,"Indianapolis, IN",940,1130,N,110,1,427
31473,2020-01-31,AA,AA_CODESHARE,AA,4399,YX,N128HQ,4399,12339,IND,"Indianapolis, IN",11057,CLT,"Charlotte, NC",1202,1344,N,102,1,427
31474,2020-01-31,AA,AA_CODESHARE,AA,4400,YX,N110HQ,4400,11278,DCA,"Washington, DC",12339,IND,"Indianapolis, IN",1940,2137,N,117,1,500


### Delay Averages from flights dataset

In [183]:
carrier_delay = {'WN': 3.727365467992859, 'DL': 3.8649281176863926, 'MQ': 1.7425742574257426, 'YV': 4.975285171102661, 
                'PT': 2.802919708029197, 'UA': 3.761437908496732, 'OO': 4.858203916272789, 'EV': 10.87032967032967, 
                'G7': 8.091286307053942, 'YX': 3.2358490566037736, '9E': 3.2622739018087854, 'G4': 9.703703703703704, 
                'AA': 4.564059339177343, 'CP': 3.3344947735191637, 'QX': 1.847457627118644, 'NK': 3.289568345323741, 
                'AX': 6.3875, 'HA': 2.65625, 'B6': 5.428360413589365, 'OH': 3.723849372384937, 'AS': 3.430513595166163,
                'F9': 7.009685230024213, 'ZW': 3.430513595166163, 'C5': 4.96039603960396, 'EM': 0, 'KS': 0, 'VX': 0.0}



weather_delay = {'WN': 0.19459321601632237, 'DL': 0.6084921430959546, 'MQ': 0.7564356435643564, 'YV': 0.10836501901140684,
               'PT': 0.42700729927007297, 'UA': 0.6606753812636166, '00': 1.1201890614449697, 'EV':0.18021978021978022,
               'G7': 0.966804979253112, 'YX': 0.6446540880503144, '9E': 0.3733850129198966, 'G4': 0.0,
               'AA': 0.5906945380984491, 'CP': 0.11498257839721254, 'QX': 0.19322033898305085, 'NK': 0.5755395683453237,
               'AX': 4.129166666666666, 'HA': 0.0, 'B6': 0.2895125553914328, 'OH': 1.1192468619246863, 'AS': 0.30966767371601206,
               'F9': 0.2687651331719128, 'ZW': 0.176056338028169, 'C5': 4.693069306930693, 'EM': 0, 'KS':0, 'VX':0.0}


nas_delay = {'WN' :1.8834480999744962, 'DL': 2.809762621196924, 'MQ':4.330693069306931, 'YV':2.693916349809886,
             'PT':4.321167883211679, 'UA':3.8649237472766886, '00':2.37272113436867, 'EV':3.024175824175824,
             'G7':1.7593360995850622, 'YX':3.256289308176101, '9E':3.5271317829457365, 'G4':1.4444444444444444,
             'AA':3.4265003371544167, 'CP':1.2299651567944252, 'QX':1.464406779661017, 'NK':7.76978417266187,
             'AX': 3.2708333333333335, 'HA':0.0, 'B6': 5.908419497784343, 'OH': 2.192468619246862, 'AS':4.069486404833837,
             'F9':3.154963680387409, 'ZW': 1.3943661971830985, 'C5':9.554455445544555, 'EM':0, 'KS':0, 'VX':0.75}


security_delay = {'WN': 0.015302218821729151, 'DL': 0.0, 'MQ':0.0, 'YV':0.0,
                  'PT': 0.0, 'UA': 0.009259259259259259, 'OO': 0.0, 'EV':0.0,
                  'G7': 0.0, 'YX': 0.031446540880503145, '9E': 0.0, 'G4': 0.5802469135802469,
                  'AA': 0.01921780175320297, 'CP': 0.0, 'QX': 0.0, 'NK': 0.0,
                  'AX': 0.0, 'HA': 0.0, 'B6': 0.0103397341211226, 'OH':0.0, 'AS':0.1540785498489426,
                  'F9': 0.0, 'ZW':0.0, 'C5':0.0, 'EM':0, 'KS': 0, 'VX':0.0}

late_aircraft_delay = {'WN': 4.490946187197143, 'DL':2.885322634570378, 'MQ':3.6653465346534655, 'YV':8.260456273764259,
                       'PT':5.142335766423358, 'UA':6.215686274509804, 'OO':5.638082376772451, 'EV':5.367032967032967,
                       'G7':5.668049792531121, 'YX':4.897798742138365, '9E':4.640826873385013, 'G4':2.197530864197531,
                       'AA':5.848617666891436, 'CP':3.9686411149825784, 'QX':1.5728813559322035, 'NK':3.264388489208633,
                       'AX':13.533333333333333, 'HA':0.0, 'B6':7.327917282127031, 'OH':6.688284518828452, 'AS':3.8716012084592144,
                       'F9':8.467312348668282, 'ZW':4.47887323943662, 'C5':15.574257425742575, 'EM':0, 'KS':0, 'VX':10.75}


dep_delay = {'WN':11.87554195358327, 'DL':8.658308258107656, 'MQ':4.9504950495049505, 'YV':13.098859315589353,
             'PT': 7.390510948905109, 'UA':11.708605664488017, 'OO':10.525995948683322, 'EV':16.334065934065933,
             'G7': 14.282157676348548, 'YX': 7.570754716981132, '9E':7.760981912144703, 'G4':11.765432098765432,
             'AA': 12.260283209710048, 'CP':5.7282229965156795, 'QX':1.806779661016949, 'NK':12.226618705035971,
             'AX':22.5375, 'HA':-0.875, 'B6': 16.04431314623338, 'OH': 10.828451882845188, 'AS':7.329305135951661,
             'F9': 17.406779661016948, 'ZW':3.2816901408450705, 'C5':28.06930693069307, 'EM':0, 'KS':0, 'VX':9.5}

### Add delay averages to the dataframe (obtained from flights dataset)

In [184]:
df['avg_carrier_delays'] = 0

for word in carrier_delay.keys():
    df.loc[df['op_unique_carrier'].str.contains(word, na=False), 
                   ['avg_carrier_delays']] = carrier_delay[word]

In [185]:
df['avg_weather_delays'] = 0

for word in weather_delay.keys():
    df.loc[df['op_unique_carrier'].str.contains(word, na=False), 
                   ['avg_weather_delays']] = weather_delay[word]

In [186]:
df['avg_nas_delays'] = 0

for word in nas_delay.keys():
    df.loc[df['op_unique_carrier'].str.contains(word, na=False), 
                   ['avg_nas_delays']] = nas_delay[word]

In [187]:
df['avg_security_delays'] = 0

for word in security_delay.keys():
    df.loc[df['op_unique_carrier'].str.contains(word, na=False), 
                   ['avg_security_delays']] = security_delay[word]

In [188]:
df['avg_late_aircraft_delays'] = 0

for word in late_aircraft_delay.keys():
    df.loc[df['op_unique_carrier'].str.contains(word, na=False), 
                   ['avg_late_aircraft_delays']] = late_aircraft_delay[word]

In [189]:
df['avg_dep_delays'] = 0

for word in dep_delay.keys():
    df.loc[df['op_unique_carrier'].str.contains(word, na=False), 
                   ['avg_dep_delays']] = dep_delay[word]

In [190]:
df.shape

(31476, 26)

### Import weather data

In [191]:
df_weather = pd.read_csv('weather_dataframe.csv')

In [192]:
df_weather

,fl_date,temp_night_list,temp_day_list,speed_night,speed_day,desc_night,desc_day,origin_city_name
0,2018-01-01,-17,-17,21,15,Clear,Patchy light snow,"Chicago, IL"
1,2018-01-02,-18,-14,15,19,Clear,Partly cloudy,"Chicago, IL"
2,2018-01-03,-13,-9,26,19,Clear,Moderate snow,"Chicago, IL"
3,2018-01-04,-13,-12,23,18,Partly cloudy,Sunny,"Chicago, IL"
4,2018-01-05,-14,-13,15,18,Clear,Sunny,"Chicago, IL"
...,...,...,...,...,...,...,...,...
27355,2020-06-26,26,28,21,22,Partly cloudy,Overcast,"Dallas/Fort Worth, TX"
27356,2020-06-27,26,31,26,23,Partly cloudy,Partly cloudy,"Dallas/Fort Worth, TX"
27357,2020-06-28,28,31,25,28,Partly cloudy,Cloudy,"Dallas/Fort Worth, TX"
27358,2020-06-29,28,31,29,30,Partly cloudy,Thundery outbreaks possible,"Dallas/Fort Worth, TX"


### Convert weather descriptions into categories

In [193]:
df_weather['weather_desc'] = ''

weather_dict = {'Patchy light snow': 'Snow', 'Overcast': 'Cloudy', 'Cloudy':'Cloudy', 'Light rain':'Rain',
       'Sunny':'Sunny', 'Partly cloudy':'Cloudy', 'Moderate snow':'Snow', 'Patchy light rain':'Rain',
       'Light drizzle':'Cloudy', 'Patchy rain possible':'Cloudy', 'Patchy light drizzle':'Cloudy',
       'Fog':'Cloudy', 'Blizzard':'Storm', 'Heavy rain':'Rain', 'Heavy snow':'Snow', 'Light rain shower':'Rain',
       'Moderate rain':'Rain', 'Mist':'Cloudy', 'Patchy heavy snow':'Snow',
       'Moderate or heavy rain shower':'Rain', 'Thundery outbreaks possible':'Storm',
       'Patchy light rain with thunder':'Rain', 'Torrential rain shower':'Rain',
       'Light snow':'Snow', 'Heavy rain at times':'Rain', 'Moderate rain at times':'Rain',
       'Patchy moderate snow':'Snow', 'Light sleet':'Snow', 'Moderate or heavy sleet':'Snow',
       'Light snow showers':'Snow', 'Moderate or heavy snow showers':'Rain',
       'Light freezing rain':'Storm', 'Moderate or heavy rain with thunder':'Rain',
       'Patchy snow possible':'Snow', 'Moderate or heavy freezing rain':'Rain',
       'Freezing fog':'Cloudy'
}

for word in weather_dict.keys():
    df_weather.loc[df_weather['desc_day'].str.contains(word, na=False), 
                   ['weather_desc']] = weather_dict[word]

### Drop un-needed weather data

In [194]:
df_weather.drop(['temp_night_list', 'speed_night', 'desc_night', 'desc_day'], axis=1, inplace=True)

In [195]:
df_weather

,fl_date,temp_day_list,speed_day,origin_city_name,weather_desc
0,2018-01-01,-17,15,"Chicago, IL",Snow
1,2018-01-02,-14,19,"Chicago, IL",Cloudy
2,2018-01-03,-9,19,"Chicago, IL",Snow
3,2018-01-04,-12,18,"Chicago, IL",Sunny
4,2018-01-05,-13,18,"Chicago, IL",Sunny
...,...,...,...,...,...
27355,2020-06-26,28,22,"Dallas/Fort Worth, TX",Cloudy
27356,2020-06-27,31,23,"Dallas/Fort Worth, TX",Cloudy
27357,2020-06-28,31,28,"Dallas/Fort Worth, TX",Cloudy
27358,2020-06-29,31,30,"Dallas/Fort Worth, TX",Storm


### Rename Columns

In [196]:
df_weather.rename(columns={"speed_day": "wind_speed", "temp_day_list": "temp"}, inplace=True)

### Merge flights_test and weather dataframe - based on fl_date and origin city

In [197]:
X = pd.merge(df, df_weather, how='left', on=['fl_date','origin_city_name'])

In [198]:
X.shape

(31476, 29)

### Add 'Day of the week' column

In [199]:
X['fl_date'] = pd.to_datetime(X['fl_date'])

In [200]:
X['day_of_week'] = X['fl_date'].dt.day_name()

In [201]:
X.shape

(31476, 30)

### Looking for Null Values

In [202]:
X.isnull().sum()

fl_date                         0
mkt_unique_carrier              0
branded_code_share              0
mkt_carrier                     0
mkt_carrier_fl_num              0
op_unique_carrier               0
tail_num                      100
op_carrier_fl_num               0
origin_airport_id               0
origin                          0
origin_city_name                0
dest_airport_id                 0
dest                            0
dest_city_name                  0
crs_dep_time                    0
crs_arr_time                    0
dup                             0
crs_elapsed_time                0
flights                         0
distance                        0
avg_carrier_delays              0
avg_weather_delays              0
avg_nas_delays                  0
avg_security_delays             0
avg_late_aircraft_delays        0
avg_dep_delays                  0
temp                        10380
wind_speed                  10380
weather_desc                10380
day_of_week   

### Drop empty rows

In [204]:
X['temp'] = X['temp'].fillna('none')
X['wind_speed'] = X['wind_speed'].fillna('none')
X['weather_desc'] = X['weather_desc'].fillna('none')

for index, row in X.iterrows():   # drop rows missing weather data
    if row['temp'] == 'none':
        X.drop(index=index, inplace=True)

In [ ]:
for index, row in X.iterrows():   # drop empty rows
    if row['wind_speed'] == 'none':
        X.drop(index=index, inplace=True)

In [206]:
for index, row in X.iterrows():   # drop empty rows
    if row['weather_desc'] == 'none':
        X.drop(index=index, inplace=True)

In [223]:
X.shape

(21096, 31)

In [209]:
X.to_csv('flights_test_data.csv', index=False)

In [211]:
X.dtypes

fl_date                     datetime64[ns]
mkt_unique_carrier                  object
branded_code_share                  object
mkt_carrier                         object
mkt_carrier_fl_num                   int64
op_unique_carrier                   object
tail_num                            object
op_carrier_fl_num                    int64
origin_airport_id                    int64
origin                              object
origin_city_name                    object
dest_airport_id                      int64
dest                                object
dest_city_name                      object
crs_dep_time                         int64
crs_arr_time                         int64
dup                                 object
crs_elapsed_time                     int64
flights                              int64
distance                             int64
avg_carrier_delays                 float64
avg_weather_delays                 float64
avg_nas_delays                     float64
avg_securit

In [212]:
X['tail_num'] = X['tail_num'].fillna(0)

In [216]:
X.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance',
       'avg_carrier_delays', 'avg_weather_delays', 'avg_nas_delays',
       'avg_security_delays', 'avg_late_aircraft_delays', 'avg_dep_delays',
       'temp', 'wind_speed', 'weather_desc', 'day_of_week'],
      dtype='object')

### Add Carrier Names

In [218]:
X['carrier_name'] = ''

my_dict ={
    'UA':'United Airlines',
    'PT':'Piedmont Airlines',
    'G7':'Lindbergh Airlines',
    'CP':'Compass Airlines',
    'QX':'Horizon Airlines',
    'AX':'Trans States Airlines',
    'ZW':'Air Wisconsin',
    'C5':'CommutAir',
    'EM':'Anderson Aviation',
    'KS':'Peninsula Airways',
    'AS':'Alaska Airlines',
    '9E':'Endeavor Air',
    'B6':'JetBlue Airways',
    'EV':'ExpressJet',
    'F9':'Frontier Airlines',
    'G4':'Allegiant Air',
    'HA':'Hawaiian Airlines',
    'MQ':'Envoy Air',
    'NK':'Spirit Airlines',
    'OH':'PSA Airlines',
    'OO':'SkyWest Airlines',
    'VX':'Virgin America',
    'WN':'Southwest Airlines',
    'YV':'Mesa Airline',
    'YX':'Republic Airways',
    'AA':'American Airlines',
    'DL':'Delta Airlines'
}

for word in my_dict.keys():
    X.loc[X['op_unique_carrier'].str.contains(word, na=False), 
                   ['carrier_name']] = my_dict[word]

### Separate Coninuous and Categorical data for scaling

In [219]:
X_continuous = X.loc[:, ['temp', 'wind_speed', 'avg_carrier_delays', 'avg_weather_delays', 'avg_nas_delays', 
                                     'avg_security_delays', 'avg_late_aircraft_delays', 'avg_dep_delays']]

X_categorical = X.loc[:, ['origin_city_name', 'carrier_name', 'weather_desc', 'day_of_week']]

### Scale Data

In [220]:
scaler = StandardScaler()
scaler.fit(X_continuous)
scaled_df = scaler.transform(X_continuous)

In [221]:
X_df = pd.DataFrame(scaled_df)

X_df.columns = ['temp', 'wind_speed', 'avg_carrier_delays', 'avg_weather_delays', 'avg_nas_delays', 
                                     'avg_security_delays', 'avg_late_aircraft_delays', 'avg_dep_delays']

In [222]:
X_df.shape

(21096, 8)

### Joining coninuous and categorical variables

In [224]:
X_categorical.insert(0, 'New_ID', range(0, 0 + len(X_categorical)))
X_df.insert(0, 'New_ID', range(0, 0 + len(X_df)))

In [225]:
df_final = pd.merge(X_categorical, X_df, how='inner', on='New_ID')

In [229]:
df_final.drop(['New_ID'], axis=1, inplace=True)

In [230]:
df_final.shape

(21096, 12)

### One Hot Encoding categorical variables

In [231]:
X = pd.get_dummies(df_final)

In [232]:
X.shape

(21096, 75)

In [235]:
X.to_csv('flights_test_submission.csv')

In [236]:
X.columns

Index(['temp', 'wind_speed', 'avg_carrier_delays', 'avg_weather_delays',
       'avg_nas_delays', 'avg_security_delays', 'avg_late_aircraft_delays',
       'avg_dep_delays', 'origin_city_name_Atlanta, GA',
       'origin_city_name_Baltimore, MD', 'origin_city_name_Boston, MA',
       'origin_city_name_Charlotte, NC', 'origin_city_name_Chicago, IL',
       'origin_city_name_Dallas, TX', 'origin_city_name_Dallas/Fort Worth, TX',
       'origin_city_name_Denver, CO', 'origin_city_name_Detroit, MI',
       'origin_city_name_Fort Lauderdale, FL', 'origin_city_name_Houston, TX',
       'origin_city_name_Las Vegas, NV', 'origin_city_name_Los Angeles, CA',
       'origin_city_name_Miami, FL', 'origin_city_name_Minneapolis, MN',
       'origin_city_name_New Orleans, LA', 'origin_city_name_New York, NY',
       'origin_city_name_Newark, NJ', 'origin_city_name_Orlando, FL',
       'origin_city_name_Philadelphia, PA', 'origin_city_name_Phoenix, AZ',
       'origin_city_name_Portland, OR', 'origin_